In [ ]:
# pip , package installer for python, written in python. basic caching.
# pip install pandas
# python -m venv venv
# venv\scripts\activate

# uv , Astral created this modern package manager for python, written in rust. global caching. built in virtual environment management.
# pip install uv
# uv venv venv
# venv\scripts\activate (still need to activate the virtual environment created by uv)
# uv pip install package
# uv pip freeze > requirements.txt
# uv pip install -r requirements.txt

# after doing 'uv venv .venv, then even if you did not activate .venv\scripts\activate; when you do 'uv pip install package1 package2 etc' it automatically installs within the .venv (BUT SOMETIMES IT DOES NOT WORK UNTIL YOU ACTIVATE VENV..NEED TO INVESTIGATE)
# but to use those in your code, you still need to activate the .venv manually..

In [1]:
def find_user(uid):
    users={1:"a", 2:"b"}
    return users.get(uid)



In [5]:
print(find_user(1))
print(find_user(10))

a
None


# so actually no type hints are required while passing params or returning values from a python function
but just for documentation or clarifying to readers developers add type hints like this
def find_user(uid: int) -> Optional[str]
to say that it accepts int and returns str but if it can't find a match it can retrun None, So they choose Optional ..and because they wrote Optional, it is from the library typing so need to import that as from typing import Optional

# if you dont write type hint such as Optional[str], someone using your code can assume it will always return str and their code might fail when you return None.. so thats the key usecase.

[1,2,3] type hint for this is a List[int]
{"abcd": 1.2, "ass": 2.5} typehint for this is a Dict[str,float]


## class
class defines a custom type; it is a way to group data and behavior together
init sets up its initial state when you create an object

In [9]:
global_var = 10 # can be accessed anywhere in this file
class ClassName:
    class_var=1 # shared across all instances
    def __init__(self, instance_var):
        # this is the constructor method that initializes instance variables
        # all self. variables are instance variables
        # all non self. variables are local to this method
        self.instance_var = instance_var # unique to each instance
        local_var=5 # can be accessed only within this method. notice no self. before it.
        self.instance_var2 = local_var*instance_var
        # To access or modify class variables inside instance methods, use ClassName.class_var
        ClassName.class_var+=1
        #to modify the global variable inside a function or method, you must use the global keyword
        global global_var 
        global_var+=1

    def get_instance_var(self):
        # used to access instance variables
        # To call an instance method, use instance.method_name()
        return self.instance_var

#To define a class method, use the @classmethod decorator and pass cls as the first parameter
    @classmethod
    def get_class_var(cls):
        # used when you need to access or modify class variables
        # To call a class method, use ClassName.method_name() or instance.method_name()
        return cls.class_var
    
#To define a static method, use the @staticmethod decorator    
    @staticmethod
    def static_method(category):
        # used when the method logic is related to the class but it does not access class or instance variables
        # so that it can be called without creating an instance. and all related logic is encapsulated within the class
        # To call a static method, use ClassName.method_name() or instance.method_name()
        category_codes = {
            "A": 1,
            "B": 2,
            "C": 3
        }
        return category_codes.get(category, 0)

obj1 = ClassName(10)
print(obj1.get_instance_var())
print(ClassName.get_class_var())
print(ClassName.static_method("A"))
print(ClassName.static_method("D"))


10
2
1
0


In [6]:
# uv pip install pydantic, pydantic[email]
from pydantic import BaseModel, Field, EmailStr,field_validator, model_validator

from datetime import datetime, date, time
from typing import List, Optional
from enum import Enum

class User(BaseModel):
    # class variables with type annotations
    age: int = Field( ge=0, le=150)
    name: str
    email: EmailStr = Field(..., description="must be valid email address")
    ssn: str = Field(..., pattern=r"^\d{3}-\d{2}-\d{4}$", description="must be valid SSN")
    is_active: bool = True
    signup_ts: datetime = None
    friends: list[int] = Field(default_factory=list) 
    # default factory is used to create a new list for each instance

user = User(age=30, name="John Doe", email="john.doe@example.com", ssn="123-45-6789")
print(user)


age=30 name='John Doe' email='john.doe@example.com' ssn='123-45-6789' is_active=True signup_ts=None friends=[]


# above output confirms that if the variables have all values as per pydantic rules we defined, then it does not give any errors.

In [7]:
user = User(age=0, name="Jane Smith", email="example.com", ssn="987-65-4321")
print(user)

ValidationError: 1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='example.com', input_type=str]

# so we were able to validate email address format as shown in above error message

In [8]:
user = User(age=0, name="Jane Smith", email="ex@ample.com", ssn="9a87-65-4321")
print(user)

ValidationError: 1 validation error for User
ssn
  String should match pattern '^\d{3}-\d{2}-\d{4}$' [type=string_pattern_mismatch, input_value='9a87-65-4321', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_pattern_mismatch

# so we were able to validate if a pattern is being followed for a variable value as per above error message.

In [9]:
import re
class Class2(BaseModel):
    password: str = Field(..., min_length=8, description="must be at least 8 characters long")
    
    @field_validator('password')
    @classmethod
    def validate_password(cls, value):
        if not re.search(r'[A-Z]', value):
            raise ValueError("Password must contain at least one uppercase letter")
        if not re.search(r'[a-z]', value):
            raise ValueError("Password must contain at least one lowercase letter")
        if not re.search(r'[0-9]', value):
            raise ValueError("Password must contain at least one digit")
        if not re.search(r'[@$!%*?&]', value):
            raise ValueError("Password must contain at least one special character")
        return value
obj = Class2(password="WeakPass123!")
print(obj)

password='WeakPass123!'


In [13]:
obj = Class2(password="abEd1efgh")
print(obj)

ValidationError: 1 validation error for Class2
password
  Value error, Password must contain at least one special character [type=value_error, input_value='abEd1efgh', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

# SO we can use field_validator to do field level detailed validations as shown above. these are not just pattern validations but also business logic validations using regex etc.

In [ ]:
from typing import Dict
# test_cases is a list of dictionaries 
# each test case has a description and data with the password to be tested
test_cases = [
    {'description': 'Valid strong password',
    'data': {'password': 'StrongP@ss1'}
    },
    {'description': 'Missing uppercase letter',
    'data': {'password': 'strongp@ss1'}
    },
    {'description': 'Missing lowercase letter',
    'data': {'password': 'STRONGP@SS1'}
    }
]
for case in test_cases: #for each item in the list
    print(f"Test Case: {case['description']}") # each item is a dictionary, so show the key's value
    try:
        # ** unpacks the dictionary to pass as keyword arguments
        obj = Class2(**case['data'])
        print("  Passed:", obj)
    except ValueError as e:
        print("  Failed:", e)
# **case['data'] unpacks the data dictionary to pass as keyword arguments to Class2
# *case['data'] would unpack as positional arguments, which is not suitable here


Test Case: Valid strong password
  Passed: password='StrongP@ss1'
Test Case: Missing uppercase letter
  Failed: 1 validation error for Class2
password
  Value error, Password must contain at least one uppercase letter [type=value_error, input_value='strongp@ss1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
Test Case: Missing lowercase letter
  Failed: 1 validation error for Class2
password
  Value error, Password must contain at least one lowercase letter [type=value_error, input_value='STRONGP@SS1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [ ]:
class Class3(BaseModel):
    birth_date: date = Field(..., description="User's birth date")
    age: int = Field(..., ge=0, le=150, description="User's age in years")

    # validates multiple fields together after object creation
    # why we pass cls and self: cls is the class itself, self is the instance being validated
    @model_validator(mode='after')
    def check_age_consistency(cls, self):
        today = date.today()
        calculated_age = today.year - self.birth_date.year - (
            (today.month, today.day) < (self.birth_date.month, self.birth_date.day)
        )
        if calculated_age != self.age:
            raise ValueError(f"Age {self.age} does not match birth date {self.birth_date}")
        return self
test_cases2= [
    {'description': 'Consistent age and birth date',
    'data': {'birth_date': date(1980, 1, 1), 'age': 45}
    },
    {'description': 'Inconsistent age and birth date',
    'data': {'birth_date': date(2000, 1, 1), 'age': 22}
    }
]
for case in test_cases2:
    print(f"Test Case: {case['description']}")
    try:
        obj = Class3(**case['data'])
        print("  Passed:", obj)
    except ValueError as e:
        print("  Failed:", e)

Test Case: Consistent age and birth date
  Passed: birth_date=datetime.date(1980, 1, 1) age=45
Test Case: Inconsistent age and birth date
  Failed: 1 validation error for Class3
  Value error, Age 22 does not match birth date 2000-01-01 [type=value_error, input_value={'birth_date': datetime.d...(2000, 1, 1), 'age': 22}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


C:\Users\sarad\AppData\Local\Temp\ipykernel_10448\126566644.py:4: PydanticDeprecatedSince212: Using `@model_validator` with mode='after' on a classmethod is deprecated. Instead, use an instance method. See the documentation at https://docs.pydantic.dev/2.12/concepts/validators/#model-after-validator. Deprecated in Pydantic V2.12 to be removed in V3.0.
  @model_validator(mode='after')


# so using model_validator, after the object is created, we can cross validate multiple fields of the object 

In [ ]:
list1=[1,2,3]
print(list1[-1]) # last item on teh list
print(list1[0]) # first item on the list

3
1


In [ ]:
j={"k1": "v1", "k2": "v2"}
import json
d=json.loads(j) # deserializes a JSON string into a Python dictionary
print(d)

i=json.dumps(d)
print(i) # serializes a Python dictionary into a JSON string

# example of using json.loads and json.dumps


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
# demo of json.loads and json.dumps
#!/usr/bin/env python3
import json
j = '{"k1": "v1", "k2": "v2"}'
d = json.loads(j)  # deserializes a JSON string into a Python dictionary
print(d)


In [ ]:
#def loads(s, *, cls=None, object_hook=None, parse_float=None,
        parse_int=None, parse_constant=None, object_pairs_hook=None, **kw):
"""
s is string 
* means all following parameters are keyword-only
all other params such as cls, object_hook etc =None means they are optional by default.
**kw is to capture any additional keyword arguments not explicitly defined
"""